In [23]:
import pandas as pd
import sqlite3

In [30]:
conn = sqlite3.connect('test.db')

In [31]:
base = pd.read_sql('select * from gri_base', conn)
base = base.drop_duplicates()

In [32]:
print(base.shape)
base.head()

(14911, 7)


,ORG_name,ORG_url,size,sector,country,Region,Reports
0,104,/organizations/8738/,MNE,Media,/static/img/flag-icons/flags_iso/16/tw.png,Asia,None
1,1&1 Drillisch AG,/organizations/18810/,Large,Telecommunications,/static/img/flag-icons/flags_iso/16/de.png,Europe,None
2,123,/organizations/17892/,SME,Agriculture,/static/img/flag-icons/flags_iso/16/af.png,Asia,None
3,1time,/organizations/4821/,Large,Logistics,/static/img/flag-icons/flags_iso/16/za.png,Africa,None
4,20km de Lausanne,/organizations/15230/,SME,Tourism/Leisure,/static/img/flag-icons/flags_iso/16/ch.png,Europe,None


In [33]:
flag = pd.read_excel('flag_country.xlsx')
flag

,flag_url,country
0,/static/img/flag-icons/flags_iso/16/tw.png,Taiwan
1,/static/img/flag-icons/flags_iso/16/de.png,Germany
2,/static/img/flag-icons/flags_iso/16/af.png,Afghanistan
3,/static/img/flag-icons/flags_iso/16/za.png,South Africa
4,/static/img/flag-icons/flags_iso/16/ch.png,Switzerland
...,...,...
138,/static/img/flag-icons/flags_iso/16/ga.png,Gabon
139,/static/img/flag-icons/flags_iso/16/cm.png,Cameroon
140,/static/img/flag-icons/flags_iso/16/sr.png,Suriname
141,/static/img/flag-icons/flags_iso/16/sy.png,Syrian Arab Republic


In [34]:
rep = pd.read_sql('select * from gri_reports', conn)
rep = rep.drop_duplicates()

In [35]:
rep.shape

(62929, 4)

In [36]:
rep.head()

,ORG_url,report_url,report_year,report_label
0,/organizations/8738/,/reports/78733/,2019,GRI - Standards
1,/organizations/8738/,/reports/61008/,2018,GRI - G4
2,/organizations/8738/,/reports/56573/,2017,GRI - G4
3,/organizations/8738/,/reports/40587/,2016,Citing - GRI
4,/organizations/8738/,/reports/27981/,2015,GRI - G3.1


In [37]:
conn2 = sqlite3.connect('gri.db')

In [38]:
base.to_sql(name='gri_base',con=conn2,if_exists='append',index=False)
rep.to_sql(name='gri_reports',con=conn2,if_exists='append',index=False)
flag.to_sql(name='gri_flag',con=conn2,if_exists='append',index=False)

In [43]:
base2 = base.copy()
base2 = base2.rename(columns={'country':'country_url'})
base2 = base2.merge(flag,left_on = 'country_url',right_on='flag_url')
base2 = base2.dropna(subset=['country'])
base2.drop(columns='Reports',inplace=True)

In [44]:
base2.shape

(14911, 8)

In [45]:
base2.head()

,ORG_name,ORG_url,size,sector,country_url,Region,flag_url,country
0,104,/organizations/8738/,MNE,Media,/static/img/flag-icons/flags_iso/16/tw.png,Asia,/static/img/flag-icons/flags_iso/16/tw.png,Taiwan
1,AAEON Technology Inc.,/organizations/22189/,Large,Computers,/static/img/flag-icons/flags_iso/16/tw.png,Asia,/static/img/flag-icons/flags_iso/16/tw.png,Taiwan
2,"Ablerex Electronics Co., Ltd.",/organizations/12590/,MNE,Technology Hardware,/static/img/flag-icons/flags_iso/16/tw.png,Asia,/static/img/flag-icons/flags_iso/16/tw.png,Taiwan
3,AcBel Polytech Inc.,/organizations/8254/,MNE,Technology Hardware,/static/img/flag-icons/flags_iso/16/tw.png,Asia,/static/img/flag-icons/flags_iso/16/tw.png,Taiwan
4,Accton Technology Corp.,/organizations/17257/,MNE,Automotive,/static/img/flag-icons/flags_iso/16/tw.png,Asia,/static/img/flag-icons/flags_iso/16/tw.png,Taiwan


In [46]:

base2.to_excel('gri_base_all.xlsx',index=False)

In [52]:
base2.to_sql(name='gri_base_with_country',con=conn2,if_exists='append',index=False)

In [47]:
rep2 = rep.merge(base2[['ORG_name','ORG_url']],on='ORG_url')
rep2 = rep2.dropna(subset=['ORG_name'])


In [48]:
rep2.shape

(62929, 5)

In [49]:
rep2.head()

,ORG_url,report_url,report_year,report_label,ORG_name
0,/organizations/8738/,/reports/78733/,2019,GRI - Standards,104
1,/organizations/8738/,/reports/61008/,2018,GRI - G4,104
2,/organizations/8738/,/reports/56573/,2017,GRI - G4,104
3,/organizations/8738/,/reports/40587/,2016,Citing - GRI,104
4,/organizations/8738/,/reports/27981/,2015,GRI - G3.1,104


In [50]:
rep2.to_excel('gri_reports_all.xlsx',index=False)

In [16]:
tmp = base.drop_duplicates(subset=['country'])[['ORG_url','country']]
tmp.reset_index(inplace=True)

In [17]:
tmp

,index,ORG_url,country
0,0,/organizations/8738/,/static/img/flag-icons/flags_iso/16/tw.png
1,1,/organizations/18810/,/static/img/flag-icons/flags_iso/16/de.png
2,2,/organizations/17892/,/static/img/flag-icons/flags_iso/16/af.png
3,3,/organizations/4821/,/static/img/flag-icons/flags_iso/16/za.png
4,4,/organizations/15230/,/static/img/flag-icons/flags_iso/16/ch.png
...,...,...,...
138,12684,/organizations/16767/,/static/img/flag-icons/flags_iso/16/ga.png
139,12690,/organizations/16764/,/static/img/flag-icons/flags_iso/16/cm.png
140,13164,/organizations/21644/,/static/img/flag-icons/flags_iso/16/sr.png
141,13275,/organizations/5402/,/static/img/flag-icons/flags_iso/16/sy.png


# flag table

In [20]:
from urllib.parse import urljoin


In [21]:
domain_url = 'https://database.globalreporting.org/'
for r in tmp.itertuples():
    url = getattr(r,'ORG_url')
    url = urljoin(domain_url,url)
    print(url)
    break

https://database.globalreporting.org/organizations/8738/


In [22]:
tmp.to_csv('flag_tmp.csv',index=False)